In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import seaborn as sns
import pandas as pd

import os

from analysis_variables import *  
import babar_analysis_tools as bat

In [ ]:
# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

#Josie Laptop
topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"

# At home (Bellis)
#topdir = '/home/bellis/babar_data/bnv_plambda'

# Background
filename= f"{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet"
#filename1 = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

start = time.time()

alldata = ak.from_parquet(filename)

data= alldata[alldata["spmode"]!="-999"]
signal= alldata[alldata["spmode"]=="-999"]

print(f"Took {time.time() - start} s")

print(type(data))
print(data)

In [ ]:
data["BpostFitDeltaE"]

In [ ]:
empty_hists= bat.create_empty_histograms(hist_defs)


In [ ]:
modes= np.unique(data["spmode"])

weights1= np.ones(6)

bat.fill_histograms_v2(ak.flatten(data["BpostFitDeltaE"]), empty_hists, spmodes= modes, weights= weights1)

In [ ]:
modes= np.unique(data["spmode"])

intmodes= []
for i in modes: 
    x= int(i)
    intmodes.append(x)

weights1= np.ones(6)

bat.fill_histograms_v2(ak.flatten(data["BpostFitDeltaE"]), empty_hists, spmodes= intmodes, weights= weights1)